In [1]:
from pathlib import Path

import matplotlib
from matplotlib.cm import get_cmap
from matplotlib.colors import  to_hex

from flexx import flx, event, ui
import numpy as np
import pandas as pd
from pscript import RawJS
from pscript.stubs import Math, d3, window

[I 15:19:36 flexx.app] Asset store collected 2 new modules.


In [2]:
try:
    flx.assets.associate_asset(__name__, 'https://d3js.org/d3.v5.min.js')
except ValueError:
    pass

In [3]:
DATA_ROOT = Path('..') / 'data'

In [4]:
dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])

sensor_df.to_csv('sensor_df.csv')

### Make data for plotting

In [41]:
# We join the participant name with the activity label of the row
# to get ['d1p01M-bed', 'd1p01M-bed', ...] etc.
values = sensor_df['participant'].str.cat(sensor_df['activity_label'], sep='-')

# Using shift to compare consecutive values we can obtain a boolean series
# which is `True` at the start of every activity, and `False` otherwise.
activity_start = values.shift(1) != values

# The cumsum trick converts the boolean values to integers,
# so `True` becomes `1` and `False` becomes `0`. 
# As `activity_start` is `True` at the start of every activity and `False`
# otherwise, successive activities in the data get an different (increasing)
# number:
#
#    list(pd.Series([True, False, True, True]).cumsum()) # <- [1, 1, 2, 3]
#
sensor_df['activity_block'] = activity_start.cumsum()

# Create a data structure for plotting.
# Must make sure all data types are python native.
plot_data = {
    'time_range': [sensor_df['time'].min(), sensor_df['time'].max()],
    'participants': {}
}
for participant, participant_df in sensor_df.groupby(['participant']):

    plot_data['participants'][participant] = {
        'activities': [],
        'time': [participant_df['time'].min(), participant_df['time'].max()]
    }
    for _, activity_df in participant_df.groupby(['activity_block']):
        activity = {
            'start': activity_df['time'].min(),
            'end': activity_df['time'].max(),
            'duration': activity_df['time'].max() - activity_df['time'].min(),
            'activity': int(activity_df['activity'].unique()[0]),
            'activity_label': activity_df['activity_label'].unique()[0],
            'sensor_data': {
                'time': list(activity_df['time']),
                'front': list(activity_df['front']),
                'lateral': list(activity_df['lateral']),
                'vertical': list(activity_df['vertical'])
            }
        }
        plot_data['participants'][participant]['activities'].append(activity)
            


### Create a widget for the plot

In [56]:
SENSOR_COLORS = {
  'front': '#1f77b4',
  'vertical': '#ff7f0e',
  'lateral': '#2ca02c',
}

ACTIVITY_COLORS = {
  '4': '#fed9a6',
  '1': '#b3cde3',
  '2': '#ccebc5',
  '3': '#decbe4',
};

class ActivityPlot(flx.Widget):
    
    def init(self):
        self.node.id = self.id
        window.setTimeout(self.plot, 500)
        
        participants = list(plot_data['participants'].keys())
        self._data = {
            'participants': participants,
            'participant_index': 0
        }
    
    @flx.emitter
    def key_down(self, e):
        """Overload key_down emitter to prevent browser scroll."""
        ev = self._create_key_event(e)
        if ev.key.startswith('Arrow'):
            e.preventDefault()
        return ev
    
    @flx.reaction('key_down')
    def _handle_change_participant(self, *events):
        for ev in events:
            if ev.modifiers:
                continue
            
            elif ev.key == 'ArrowRight':
                self._data['participant_index'] = ((self._data['participant_index'] + 1) % 
                    len(self._data['participants']))
                self.replot()
            elif ev.key == 'ArrowLeft':
                self._data['participant_index'] = self._data['participant_index'] - 1
                if self._data['participant_index'] < 0:
                    self._data['participant_index'] = len(self._data['participants']) - 1
                self.replot()
                
    def clear_plot(self):
        root = d3.select('#' + self.id)
        root.select('svg').remove()
    
    def replot(self):
        self.clear_plot()
        self.plot()
        
    def plot(self):
        w, h = self.size
        
        self.clear_plot()
        
        root = d3.select('#' + self.id)
        
        svg = root.append('svg').attr('width', w).attr('height', h)
        
        participant = self._data['participants'][self._data['participant_index']]
        data = plot_data['participants'][participant]
        
        x = d3.scaleLinear().domain([0, max(data['time'])]).range([0, w])
        xAxis = svg.append('g').call(d3.axisBottom(x))
        
        y = d3.scaleLinear().domain([-2, 2]).range([h, 0])
        yAxis = svg.append('g').call(d3.axisLeft(y))
        
        paths = []
        markers = []
        rectangles = []
        labels = []
        
        nactivities = len(data['activities'])
        for i, activity_data in enumerate(data['activities']):
            
            # Plot activity rectangles
            rectangle = (svg.datum(activity_data)
                    .append('rect')
                    .attr('x', lambda d: x(d['start']))
                    .attr('y', 0)
                    .attr('width', lambda d: x(d['end']) - x(d['start']))
                    .attr('height', h)
                    .attr('fill', lambda d: ACTIVITY_COLORS[d['activity']])
                    .attr('opacity', 0.333)
                    )
            rectangles.append(rectangle)
            
            # Plot activity_labels
            label = (svg.datum(activity_data)
                   .append('text')
                   .attr('x', lambda d: (x(d['end']) + x(d['start'])) / 2)
                   .attr('y',  h - (i + 0.5) / nactivities * h)
                   .attr('text-anchor', 'middle')
                   .style('font-size', 10)
                   .text(lambda d: d['activity_label']))
            labels.append(label)
       
            # Plot sensor data
            for sensor in ['front', 'lateral', 'vertical']:
                points = list(zip(activity_data['sensor_data']['time'], activity_data['sensor_data'][sensor]))
                path = (svg.append('path')
                    .datum(points)
                    .attr('fill', 'none')
                    .attr('stroke', SENSOR_COLORS[sensor])
                    .attr('stroke-width', 1.5)
                    .attr('d', d3.line().x(lambda d: x(d[0])).y(lambda d: y(d[1])) ))
                paths.append(path)
                
                marks = (svg.selectAll('dot')
                    .data(points)
                    .enter()
                    .append('circle')
                    .attr('cx', lambda d: x(d[0]))
                    .attr('cy', lambda d: y(d[1]))
                    .attr('r', 2)
                    .style('fill', SENSOR_COLORS[sensor]))
                markers.append(marks)
                
        # Add participant label
        (svg.append('text')
           .attr('x', w / 2 )
           .attr('y',  h - 10)
           .attr('text-anchor', 'middle')
           .style('font-size', 10)
           .text('{} {}/{}'.format(
               participant,
               self._data['participant_index'] + 1,
               len(self._data['participants']))))
            
            
        def update_chart():
            
            newX = d3.event.transform.rescaleX(x)
            newY = y
            xAxis.call(d3.axisBottom(newX))
            yAxis.call(d3.axisLeft(newY))
            
            for path in paths:
                path.attr('d', d3.line().x(lambda d: newX(d[0])).y(lambda d:y(d[1])))
                
            for marks in markers:
                marks.attr('cx', lambda d: newX(d[0])).attr('cy', lambda d:y(d[1]))
            
            for rectangle in rectangles:
                (rectangle.attr('x', lambda d: newX(d['start']))
                 .attr('width', lambda d: newX(d['end']) - newX(d['start'])))
            
            for label in labels:
                (label.attr('x',  lambda d: (newX(d['end']) + newX(d['start'])) / 2))

        zoom = (d3.zoom()
            .scaleExtent([0.05, 20]) 
            .extent([[0, 0], [w, h]])
            .on('zoom', update_chart))
        
        (svg.append('rect')
            .attr('width', w)
            .attr('height', h)
            .style('fill', 'none')
            .style('pointer-events', 'all')
            .attr('transform', 'translate(0,0)')
            .call(zoom))
            

In [57]:
# Save the app as a self-contained HTML page
# and then load in to jupyter using an iframe:
app = flx.App(ActivityPlot)
html = f'{app.cls.__name__}.html'
app.export(html, link=0)
from IPython.display import IFrame

IFrame(html, width=800, height=400)

[I 16:13:05 flexx.app] Exported standalone app to '/Users/mwibrow/github/motiondetection/notebooks/ActivityPlot.html'
